# Consul Failover with Prepared Queries

* **Static** lists of **alternate datacenters**
* **Dynamic** policies that make use of network **Round Trip Time (RTT)**
* **Hybrid** policies that make use of network RTT 1st then alternate DC's
* **Template** allows one prepared query to apply to many or **all services**

https://learn.hashicorp.com/tutorials/consul/automate-geo-failover

### Register a Service

In [39]:
for node_number in 0 3 4; do
docker exec -i consul-server-${node_number} sh <<EOM
printf "#==> create service definition.\n"
cat > /tmp/consul_service.json <<EOF
{
  "ID": "redis1",
  "Name": "redis",
  "Tags": ["primary", "v1"],
  "Address": "127.0.0.1",
  "Port": 8000,
  "Meta": {
    "redis_version": "4.0"
  },
  "EnableTagOverride": false
}
EOF
#   "Check": {
#     "DeregisterCriticalServiceAfter": "90m",
#     "Args": ["/usr/local/bin/check_redis.py"],
#     "Interval": "10s",
#     "Timeout": "5s"
#   },
#   "Weights": {
#     "Passing": 10,
#     "Warning": 1
#   }
# }
# EOF
printf "\n#==> register service.\n"

curl \
    --request PUT \
    --data @/tmp/consul_service.json \
    http://127.0.0.1:8500/v1/agent/service/register #?replace-existing-checks=true
EOM
done

#==> create service definition.

#==> register service.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   174    0     0  100   174      0   6449 --:--:-- --:--:-- --:--:--  6692
#==> create service definition.

#==> register service.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   174    0     0  100   174      0   8980 --:--:-- --:--:-- --:--:--  9666
#==> create service definition.

#==> register service.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   174    0     0  100   174      0  11705 --:--:-- --:--:-- --:--:-- 12428


In [10]:
curl \
    --request PUT \
    http://127.0.0.1:8500/v1/agent/service/deregister/redis1

## Static policy

A static failover policy includes a fixed list of datacenters to contact once there are no healthy instances in the local datacenter.

Here's the example from the introduction, expanded with a static failover policy:

In [1]:
export CONSUL_DC=west CONSUL_DC_2=dc2

In [53]:
curl http://127.0.0.1:8500/v1/query \
    --request POST \
    --data @- << EOF
{
  "Name": "redis-prod",
  "Service": {
    "Service": "redis",
    "Failover": {
      "Datacenters": ["${CONSUL_DC}", "${CONSUL_DC_2}"]
    }
  }
}
EOF

{"ID":"d28c8ed3-b2de-ebaa-4d8b-1a3845b89e4d"}

When this query is executed, such as with a DNS lookup to "`banking-app.query.consul`", the following actions will occur:

    Consul servers in the local datacenter will attempt to find healthy instances of the "banking-app" service with the required tag.
    If none are available locally, the Consul servers will make an RPC request to the Consul servers in "dc2" to perform the query there.
    If none are available in "dc2", then an RPC will be made to the Consul servers in "dc3" to perform the query there.
    Finally an error will be returned if none of these datacenters had any instances available.

### Verify Prepared Queries were added to the system

In [57]:
PREPARED_QUERIES=$(curl -s http://127.0.0.1:8500/v1/query | jq ".") \
  && echo $PREPARED_QUERIES | jq

[
  {
    "ID": "8d6df755-5de4-1fc7-a695-3cc6bb063f90",
    "Name": "",
    "Session": "",
    "Token": "",
    "Template": {
      "Type": "name_prefix_match",
      "Regexp": "",
      "RemoveEmptyTags": false
    },
    "Service": {
      "Service": "${name.full}",
      "Failover": {
        "NearestN": 2,
        "Datacenters": null
      },
      "OnlyPassing": false,
      "IgnoreCheckIDs": null,
      "Near": "",
      "Tags": null,
      "NodeMeta": null,
      "ServiceMeta": null,
      "Connect": false
    },
    "DNS": {
      "TTL": ""
    },
    "CreateIndex": 5524,
    "ModifyIndex": 5524
  }
]


In [51]:
dig @127.0.0.1 -p 8600 redis-prod.query.consul srv


; <<>> DiG 9.16.1-Ubuntu <<>> @127.0.0.1 -p 8600 redis-prod.query.consul srv
; (1 server found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NXDOMAIN, id: 63353
;; flags: qr aa rd; QUERY: 1, ANSWER: 0, AUTHORITY: 1, ADDITIONAL: 1
;; WARNING: recursion requested but not available

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;redis-prod.query.consul.	IN	SRV

;; AUTHORITY SECTION:
consul.			0	IN	SOA	ns.consul. hostmaster.consul. 1639160756 3600 600 86400 0

;; Query time: 7 msec
;; SERVER: 127.0.0.1#8600(127.0.0.1)
;; WHEN: Fri Dec 10 18:25:56 UTC 2021
;; MSG SIZE  rcvd: 102



### Deregister service from DC1

In [35]:
docker exec consul-server-0 \
curl \
    --request PUT \
    http://127.0.0.1:8500/v1/agent/service/deregister/redis1

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


Verify answer is now for DC2

In [36]:
dig @127.0.0.1 -p 8600 redis-prod.query.consul srv


; <<>> DiG 9.16.1-Ubuntu <<>> @127.0.0.1 -p 8600 redis-prod.query.consul srv
; (1 server found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 3120
;; flags: qr aa rd; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 4
;; WARNING: recursion requested but not available

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;redis-prod.query.consul.	IN	SRV

;; ANSWER SECTION:
redis-prod.query.consul. 0	IN	SRV	1 1 8000 7f000001.addr.dc2.consul.

;; ADDITIONAL SECTION:
7f000001.addr.dc2.consul. 0	IN	A	127.0.0.1
consul-server-3.node.dc2.consul. 0 IN	TXT	"consul-network-segment="
consul-server-3.node.dc2.consul. 0 IN	TXT	"zone=zone0"

;; Query time: 3 msec
;; SERVER: 127.0.0.1#8600(127.0.0.1)
;; WHEN: Fri Dec 10 16:59:40 UTC 2021
;; MSG SIZE  rcvd: 192



Add service back in.

In [37]:
for node_number in 0; do
docker exec -i consul-server-${node_number} sh <<EOM
printf "\n#==> register service.\n"

curl \
    --request PUT \
    --data @/tmp/consul_service.json \
    http://127.0.0.1:8500/v1/agent/service/register #?replace-existing-checks=true
EOM
done


#==> register service.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   174    0     0  100   174      0   3203 --:--:-- --:--:-- --:--:--  3283


Verify answer is now for DC1 again.

In [38]:
dig @127.0.0.1 -p 8600 redis-prod.query.consul srv


; <<>> DiG 9.16.1-Ubuntu <<>> @127.0.0.1 -p 8600 redis-prod.query.consul srv
; (1 server found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 63677
;; flags: qr aa rd; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 3
;; WARNING: recursion requested but not available

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;redis-prod.query.consul.	IN	SRV

;; ANSWER SECTION:
redis-prod.query.consul. 0	IN	SRV	1 1 8000 7f000001.addr.west.consul.

;; ADDITIONAL SECTION:
7f000001.addr.west.consul. 0	IN	A	127.0.0.1
consul-server-0.node.west.consul. 0 IN	TXT	"consul-network-segment="

;; Query time: 0 msec
;; SERVER: 127.0.0.1#8600(127.0.0.1)
;; WHEN: Fri Dec 10 17:00:56 UTC 2021
;; MSG SIZE  rcvd: 170



## Delete prepared query

In [71]:
curl -s http://127.0.0.1:8500/v1/query/$(echo $PREPARED_QUERIES | jq -r .[0].ID) -X DELETE

## Prepared query template

For datacenters with many services, it can be challenging to define a geo failover policy for each service. To relieve this challenge, Consul provides a [prepared query template](https://www.consul.io/api/query.html#prepared-query-templates) that allows one prepared query to apply to many, and even all, services.

Templates can match on prefixes or use full regular expressions to determine which services they match.

Below is an example request to create a prepared query template that applies a catch-all policy of dynamic geo failover to all services accessed by query lookup (`*.query.consul`). By specifying the `name_prefix_match` type and an empty name, this query template's policy will be applied to any name (`<name>.query.consul`) that doesn't [match a higher-precedence query](https://www.consul.io/api-docs/query#type).

```shell
$ curl http://127.0.0.1:8500/v1/query \
    --request POST \
    --data @- << EOF
{
  "Name": "",
  "Template": {
    "Type": "name_prefix_match"
  },
  "Service": {
    "Service": "${name.full}",
    "Failover": {
      "NearestN": 2
    }
  }
}
EOF
```

In [72]:
curl http://127.0.0.1:8500/v1/query \
    --request POST \
    --data @- << "EOF"
{
  "Name": "",
  "Template": {
    "Type": "name_prefix_match"
  },
  "Service": {
    "Service": "${name.full}",
    "Failover": {
      "NearestN": 2,
      "Datacenters": ["west","dc2","dc3"]
    }
  }
}
EOF

{"ID":"acd3de07-868a-bbf0-f02d-8d16a00249d2"}

Verify query was added to the system.

In [67]:
curl -s http://127.0.0.1:8500/v1/query | jq

[
  {
    "ID": "8d6df755-5de4-1fc7-a695-3cc6bb063f90",
    "Name": "",
    "Session": "",
    "Token": "",
    "Template": {
      "Type": "name_prefix_match",
      "Regexp": "",
      "RemoveEmptyTags": false
    },
    "Service": {
      "Service": "${name.full}",
      "Failover": {
        "NearestN": 2,
        "Datacenters": null
      },
      "OnlyPassing": false,
      "IgnoreCheckIDs": null,
      "Near": "",
      "Tags": null,
      "NodeMeta": null,
      "ServiceMeta": null,
      "Connect": false
    },
    "DNS": {
      "TTL": ""
    },
    "CreateIndex": 5524,
    "ModifyIndex": 5524
  }
]


In [74]:
dig @127.0.0.1 -p 8600 redis.query.consul srv


; <<>> DiG 9.16.1-Ubuntu <<>> @127.0.0.1 -p 8600 redis.query.consul srv
; (1 server found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 29570
;; flags: qr aa rd; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 3
;; WARNING: recursion requested but not available

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;redis.query.consul.		IN	SRV

;; ANSWER SECTION:
redis.query.consul.	0	IN	SRV	1 1 8000 7f000001.addr.west.consul.

;; ADDITIONAL SECTION:
7f000001.addr.west.consul. 0	IN	A	127.0.0.1
consul-server-0.node.west.consul. 0 IN	TXT	"consul-network-segment="

;; Query time: 0 msec
;; SERVER: 127.0.0.1#8600(127.0.0.1)
;; WHEN: Fri Dec 10 18:39:11 UTC 2021
;; MSG SIZE  rcvd: 165



### Deregister service from DC1

In [75]:
docker exec consul-server-0 \
curl \
    --request PUT \
    http://127.0.0.1:8500/v1/agent/service/deregister/redis1

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


Verify answer is now for DC2 or DC3

In [81]:
dig @127.0.0.1 -p 8600 redis.query.consul srv


; <<>> DiG 9.16.1-Ubuntu <<>> @127.0.0.1 -p 8600 redis.query.consul srv
; (1 server found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 62228
;; flags: qr aa rd; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 4
;; WARNING: recursion requested but not available

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;redis.query.consul.		IN	SRV

;; ANSWER SECTION:
redis.query.consul.	0	IN	SRV	1 1 8000 7f000001.addr.dc3.consul.

;; ADDITIONAL SECTION:
7f000001.addr.dc3.consul. 0	IN	A	127.0.0.1
consul-server-4.node.dc3.consul. 0 IN	TXT	"consul-network-segment="
consul-server-4.node.dc3.consul. 0 IN	TXT	"zone=zone1"

;; Query time: 3 msec
;; SERVER: 127.0.0.1#8600(127.0.0.1)
;; WHEN: Fri Dec 10 18:48:10 UTC 2021
;; MSG SIZE  rcvd: 187



Add service back in.

In [82]:
for node_number in 0; do
docker exec -i consul-server-${node_number} sh <<EOM
printf "\n#==> register service.\n"

curl \
    --request PUT \
    --data @/tmp/consul_service.json \
    http://127.0.0.1:8500/v1/agent/service/register #?replace-existing-checks=true
EOM
done


#==> register service.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   174    0     0  100   174      0   2969 --:--:-- --:--:-- --:--:--  3000


Verify answer is now for DC1 again.

In [89]:
dig @127.0.0.1 -p 8600 redis.query.consul srv


; <<>> DiG 9.16.1-Ubuntu <<>> @127.0.0.1 -p 8600 redis.query.consul srv
; (1 server found)
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 59147
;; flags: qr aa rd; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 3
;; WARNING: recursion requested but not available

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;redis.query.consul.		IN	SRV

;; ANSWER SECTION:
redis.query.consul.	0	IN	SRV	1 1 8000 7f000001.addr.west.consul.

;; ADDITIONAL SECTION:
7f000001.addr.west.consul. 0	IN	A	127.0.0.1
consul-server-0.node.west.consul. 0 IN	TXT	"consul-network-segment="

;; Query time: 3 msec
;; SERVER: 127.0.0.1#8600(127.0.0.1)
;; WHEN: Fri Dec 10 18:48:47 UTC 2021
;; MSG SIZE  rcvd: 165



> **Note**: If multiple queries are registered, the most specific one will be selected, so it's possible to have a template like this as a catch-all, and then apply more specific policies to certain services.

With this one prepared query template in place, simply changing application configurations to look up `banking-app.query.consul` instead of `banking-app.service.consul` via DNS will result in automatic geo failover to the next closest federated Consul datacenters, in order of increasing network round trip time.

## Next steps

In this tutorial, you learned how to use prepared queries for failover when integrating Consul with other applications. You can now configure your policies to failover to the nearest federated datacenter or to a list of secondary datacenters. You can also create a prepared query template which will help you reduce some complexity of creating policies for each individual service.

## Resources

https://www.devtech101.com/2019/01/30/updated-using-consul-for-service-discovery-in-multiple-data-centers-version-1-4-part-2/